In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from netam import noof, shmoof
from epam.torch_common import pick_device

In [7]:
train_df, val_df = shmoof.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv", sample_count=5000)

In [8]:
kmer_length = 5
max_length = 500

train_dataset = shmoof.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = shmoof.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

we have 3608 training examples and 1392 validation examples


In [9]:
model = noof.NoofModel(train_dataset, embedding_dim=2, nhead=2, dim_feedforward=512, layer_count=3, dropout=0.1)

# device = pick_device()
# train_dataset.to(device)
# val_dataset.to(device)
# model.to(device)

burrito = shmoof.NoofBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.5, l2_regularization_coeff=1e-6)
print("starting training...")
losses = burrito.train(epochs=10)

starting training...
Epoch [1/10]	 Loss: 0.071663097	 Val Loss: 0.069396995
Epoch [2/10]	 Loss: 0.061702187	 Val Loss: 0.069825951
Epoch [3/10]	 Loss: 0.061513994	 Val Loss: 0.069570079
Epoch [4/10]	 Loss: 0.060912887	 Val Loss: 0.068465089
Epoch [5/10]	 Loss: 0.060269527	 Val Loss: 0.06773097
Epoch [6/10]	 Loss: 0.059777284	 Val Loss: 0.067315882
Epoch [7/10]	 Loss: 0.059450003	 Val Loss: 0.066974686
Epoch [8/10]	 Loss: 0.05928794	 Val Loss: 0.066756285
Epoch [9/10]	 Loss: 0.05914167	 Val Loss: 0.066699952
Epoch [10/10]	 Loss: 0.059067863	 Val Loss: 0.06659212


In [ ]:
train_dataset[0][0]

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         382,  501,  979,  844,  303,  187,  748,  944,  701,  754,  968,  797,
         114,  456,  799,  123,  489,  931,  650,  552,  159,  635,  489,  932,
         654,  566,  215,  858,  358,  405,  595,  330,  294,  150,  598,  342,
         341,  339,  331,  299,  169,  673,  643,  523,   43,  171,  682,  680,
         671,  635,  489,  931,  652,  559,  187,  745,  932,  656,  575,  251,
        1003,  939,  681,  673,  641,  5

In [ ]:
untrained_model = noof.NoofModel(train_dataset, embedding_dim=2, nhead=2, dim_feedforward=512, layer_count=3, dropout=0.5)


#untrained_model(torch.tensor(train_dataset[0][0]).unsqueeze(0).to("cpu"))
untrained_model(torch.tensor(train_dataset[0:5][0]).to("cpu"))[:,100:105]

/var/folders/16/pkph6n2962g3lf2wpwwt0wwr0000gn/T/ipykernel_3232/838397735.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  untrained_model(torch.tensor(train_dataset[0:5][0]).to("cpu"))[:,100:105]


tensor([[ 0.0830,  0.1990,  2.3263,  1.3730,  1.6612],
        [ 0.9430,  0.1540,  0.2526,  1.3730,  1.6612],
        [ 0.0830,  0.3618,  0.1525,  0.5621,  0.9796],
        [ 0.0696,  0.9046,  1.4014,  0.2176,  1.6827],
        [ 0.9430,  0.6074,  0.2316, 16.9417,  3.7528]],
       grad_fn=<SliceBackward0>)

In [ ]:
model(torch.tensor(train_dataset[0:5][0]))[:,100:105]

/var/folders/16/pkph6n2962g3lf2wpwwt0wwr0000gn/T/ipykernel_3232/3090242575.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model(torch.tensor(train_dataset[0:5][0]))[:,100:105]


tensor([[0.6581, 0.3463, 1.5333, 6.9397, 3.4632],
        [0.4631, 0.3333, 2.1145, 6.9397, 3.4632],
        [0.6581, 0.3157, 1.7103, 2.6826, 2.5516],
        [1.4022, 2.0454, 3.5936, 2.6157, 1.5485],
        [0.4631, 0.3508, 2.3401, 5.6431, 1.8637]], grad_fn=<SliceBackward0>)

In [ ]:
torch.save(model, "_ignore/noof_model.pt")

In [ ]:
xmodel = torch.load("_ignore/noof_model.pt")
xmodel.eval()

xmodel(val_dataset.encoded_parents)[0]

tensor([0.5518, 0.6624, 0.5783, 0.5351, 0.6392, 0.6453, 0.5759, 0.5612, 0.5350,
        0.5212, 0.6106, 0.6459, 0.5476, 0.7167, 0.5565, 0.6073, 0.6326, 0.6027,
        0.6465, 0.5447, 0.5773, 0.6841, 0.6078, 0.6000, 0.6216, 0.6074, 0.5323,
        0.5783, 0.5401, 0.5363, 0.5842, 0.5510, 0.5664, 0.6187, 0.6309, 0.5810,
        0.6130, 0.5705, 0.6055, 0.6525, 0.6094, 0.6024, 0.5915, 0.5348, 1.0906,
        0.6054, 0.4977, 0.5493, 0.5011, 0.5907, 0.6027, 0.5100, 0.5653, 0.5549,
        0.4949, 0.5748, 0.5445, 0.4786, 0.5407, 0.5437, 0.5157, 0.5394, 0.4904,
        0.5689, 0.4229, 0.1278, 0.1571, 1.3315, 0.4496, 0.5732, 3.5157, 3.2419,
        0.4779, 0.4740, 0.5374, 0.6583, 0.5848, 1.9642, 1.7882, 2.4409, 1.0033,
        0.6571, 7.3481, 4.5779, 1.1166, 0.3505, 3.8996, 1.9366, 3.1124, 1.7170,
        1.9919, 8.7818, 7.5836, 2.7489, 2.4826, 2.8937, 0.8431, 2.8570, 2.4638,
        2.8824, 0.4257, 1.2694, 1.6310, 2.4068, 0.8749, 0.1843, 0.0125, 0.5162,
        0.6268, 0.4026, 0.2470, 0.1467, 

In [ ]:
val_dataset.masks[0]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 